## Connect to template

In `server.py`, set -
```
add_routes(app, chain_ext, path="/rag_ollama_multi_query")
```

In [8]:
from langserve.client import RemoteRunnable
import os
import json

def read_jsonl_files(folder_path):
    # 检查文件夹路径是否存在

    # 存储所有JSON对象的列表
    json_objects = []

    with open(folder_path, 'r', encoding='utf-8') as file:
        for line in file:
            # 解析JSON对象并添加到列表中
            json_objects.append(json.loads(line.strip()))


    return json_objects

# 文件夹路径
folder_path = 'data/multifieldqa_zh.jsonl'

# 读取JSONL文件
json_objects = read_jsonl_files(folder_path)

# 打印读取的JSON对象
for obj in json_objects:
    print(obj)
    print(obj.keys())
    print(obj['input'])
    print(obj['answers'])
    print(obj['length'])
    print(obj['context'])
    break
# rag_app_ollama = RemoteRunnable("http://127.0.0.1:8000/rag-ollama-multi-query")

# rag_app_ollama.invoke("What are the different types of agent memory?")

{'input': '全国美国文学研究会的第十八届年会在哪所大学举办的？', 'context': '全国美国文学研究会\n受秘书处委托，由我向美文会会员单位的各位代表简单汇报一下全国美国文学研究会自上届（第十七届）年会召开以来所做的工作。美文会秘书处刚刚完成了教育部社团办“关于在教育部主管社会组织中开展调研工作的通知”中要求提交的“全国美国文学研究会调研报告”（2016年11月），主要内容4项：我想就把我们提交的“调研报告”中1、2两部分中的部分内容，作为我向大会汇报的“全国美国文学研究会2014-2016年工作总结”的内容。\n1、研究会现状和基本情况\n美文会现有会员单位127个（不招收个人会员），包括国内主要985与211高校，以及中国社科院等科研单位和知名出版社。会长单位是南京大学，副会长单位是南京大学，中国社会科学院，北京外国语大学，北京大学，复旦大学，山东大学，秘书处设在南京大学外国语学院，秘书长、副秘书长是南京大学赵文书、何宁。美文会正式成立于1979年7月，是我国改革开放后成立最早的高校外国文学研究机构。1992年8月18日在民政部正式注册登记，获颁《中华人民共和国社会团体登记证》。美文会挂靠南京大学，财务由南京大学财务处负责，接受南京大学审计处审计，按民政部要求，每年参加年检，年检结果均为“通过”。\n美文会秘书处聘有专职秘书，工作人员7人，包括会长，副会长，秘书长，副秘书长，常务理事等。美文会设有党小组，隶属外国语学院英语系党支部，由会长任党小组长，成员包括副会长、常务理事、副秘书长、以及参加秘书处工作的青年教师与博士生。美文会发行《全国美国文学研究会通讯》（CASAL Newsletter），现已刊出33期。美文会每年轮流召开年会和专题研讨会，迄今已经举办17届年会和11届专题研讨会。\n2、近两年主要工作和取得的成效\n1）上届年会。美文会第十七届年会于2014年10月24日至26日在中国人民大学苏州校区举行，由中国人民大学外国语学院承办。来自全国28个省市自治区175所高等院校、研究所、出版社的348名正式代表参加了该次年会。年会的主题是“全球化语境中的美国文学研究：理论与实践”，收到论文全文164篇，摘要273篇。会议期间，全国美国文学研究会第七届理事会召开第三次会议。会议讨论通过了增补美文会副会长、常务理事、理事、会员单位事宜。会议再

In [30]:
import os
import json
import chromadb
from splitter.DynamicSizeFixedStepSplitter import DynamicSizeFixedStepSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel

output_folder = "/Users/bifenglin/Code/SW-RAG/packages/rag-ollama-multi-query/data/multifieldqa_en_bench/multifieldqa_en_answer_DFS_512_64"
input_folder = "/Users/bifenglin/Code/SW-RAG/packages/rag-ollama-multi-query/data/multifieldqa_en"
os.makedirs(output_folder, exist_ok=True)  # 创建文件夹，如果已存在则不会报错

for i in range(0, len(os.listdir(input_folder))):
    print(i)
    input_file = f"{input_folder}/output_{i}.json"
    output_file = f"{output_folder}/aoutput_{i}.json"

    with open(input_file, "r", encoding="utf-8") as file:
        json_data = json.load(file)

        # Create database client
        client = chromadb.Client()
        if i != 0:
            client.delete_collection("rag-private")

        # Split text
        text_splitter = DynamicSizeFixedStepSplitter(chunk_size=512, step_window=64)
        all_splits = text_splitter.split_text(json_data["context"])

        # Vectorize the text and store it in Chroma
        vectorstore = Chroma.from_texts(
            texts=all_splits,
            collection_name="rag-private",
            embedding=OllamaEmbeddings(),
        )
        retriever = vectorstore.as_retriever()

        # Set up RAG model
        prompt_template = """Answer the question based only on the following context:
        {context}
        Question: {question}
        """
        prompt = ChatPromptTemplate.from_template(prompt_template)


        model = ChatOllama()
        chain = (
                RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
                | prompt
                | model
                | StrOutputParser()
        )

        # Add type to input
        class Question(BaseModel):
            __root__: str

        chain = chain.with_types(input_type=Question)

        # Run the chained process and save the results to the output file
        output = {}
        try:
            output["answer"] = chain.invoke(json_data["input"])
            print(json_data["answers"])
            with open(output_file, "w", encoding="utf-8") as file:
                json.dump(output, file, ensure_ascii=False, indent=4)

        except Exception as e:
            print("Error:", e)


0
['South West Ultras fan club.']
1


KeyboardInterrupt: 